<a href="https://colab.research.google.com/github/Beauremontt/Selected-SDSS-Spectrometry/blob/main/Selected_SDSS_Spectrometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Modules and general functions

import numpy as np  # general maths

from scipy import integrate  # numerical integrations
from scipy.optimize import curve_fit  # for curve optimization

import matplotlib.pyplot as plt  # for plotting
import matplotlib.patheffects as PathEffects  # for adding text shadows

import random

from astropy.io import fits  # for FITS handling
import os  # handling file directories
from google.colab import drive  # get drive as directory
drive.mount('/content/drive')


# contants used in calculations
C = 3e6  # speed of light, km/s
H0 = 70  # Hubble constant, km/s/Mpc

In [ ]:
# @title Load spectral data

# file paths to data files
spectra_path = "/content/drive/MyDrive/SPS4020 Astrophysics 2/Project/Data/spectra_FITS"
sPlines_path = "/content/drive/MyDrive/SPS4020 Astrophysics 2/Project/Data/SDSS_spectral_lines.csv"
SDSS_search_path = "/content/drive/MyDrive/SPS4020 Astrophysics 2/Project/Data/SDSS_search.csv"


# gets list of files in folder path; returns list of relative paths
def get_files(folder_path):

  os.chdir(folder_path)  # changes directory

  files = []
  for file_name in os.listdir():  # iterates over each file in directory
    if os.path.isfile(file_name):  # checks if it's a file and not a directory
      files.append(file_name)  # add file to array

  return files


## load spectra
# create list of relative paths to spectra
spectra_files = get_files(spectra_path)
spectra_files.sort()  # sort alphabetically

print(len(spectra_files), "spectra loaded as filepaths")
print(*spectra_files, "\n")


# create list of spectra fits files
spectra_fits = [fits.open(spectrum) for spectrum in spectra_files]

print(len(spectra_fits), "spectra fits files loaded as HDU lists")


## spectral line features mentioned by the SDSS
  # basic line table mentioned in manual, https://classic.sdss.org/dr6/algorithms/linestable.php
# get array of [wavelengths, galaxy weight, quasar weight, 'species'] for the emission/absorption lines
lines = np.genfromtxt(sPlines_path, delimiter=',', dtype=(float, float, float, 'U12'), names=True)
    # columns keys are: ['wavelength', 'galaxy_weight', 'quasar_weight', 'species']

# remove break between types of lines
lines = np.delete(lines, 36)

# split lines by weight
emissions = lines[:36]
absorptions = lines[36:]

# get indices of lines for ...
# ... galaxy_weight emissions
gw_em_idx = [i for i,gw in enumerate(emissions['galaxy_weight']) if gw > 0]
# ... galaxy_weight absorptions
gw_ab_idx = range(len(absorptions))  # every saved line absorbs
# ... quasar_weight emissions (QSOs don't have absorption lines)
qw_em_idx = [i for i,qw in enumerate(emissions['quasar_weight']) if qw > 0]

# relabel the hydrogen lines
lines[1]['species'] = 'Lyα'
lines[32]['species'] = 'Hα'
lines[24]['species'] = 'Hβ'
lines[22]['species'] = 'Hγ'
lines[21]['species'] = 'Hδ'
lines[37]['species'] = 'Hε'


## SDSS search query results
  # table of results to compare IDs from HDULs to get redshifts and object classifications
  # order of specObs is the same as the sorted spectra_fits
specObs = np.genfromtxt(SDSS_search_path, delimiter=',', names=True,
                        usecols=(0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12),
                        dtype=['U4', 'U5', 'U3', 'U19', 'U12', 'U12', float, float, float, 'U2', 'U6', 'U12'])
    # columns keys: ['plate', 'mjd', 'fiberid', 'specobj_id', 'ra', 'dec', 'sn_median_r', 'z', 'zerr', 'zwarning', 'class', 'subclass']

# add redshift and class information from SDSS search query to the FITS headers
new_keys = ['ra', 'dec', 'z', 'class', 'subclass']
for spectrum, RA, DEC, Z, CLASS, SUBCLASS in zip(spectra_fits, *[specObs[key] for key in new_keys]):
  headers = spectrum[0].header  # references the headers of PrimaryHDU

  # create new dictionary entries in the header
  headers['Z'] = Z  # redshift
  headers['CLASS'] = CLASS  # object class (star, galaxy, qso)
  headers['SUBCLASS'] = SUBCLASS  # '' (starforming, starburst, broadline)

  # edit coordinate entries (the defaults still exist as RADEG and DECDEG, but they don't seem correct)
  headers['RA'] = RA  # right-ascension coordinate of target
  headers['DEC'] = DEC  # declination coordinate

In [ ]:
# @title Plot selected spectrum  { form-width: "1px" }
# you choose a single spectrum to plot
# for messing around with plot settings without waiting ~20 seconds

# index of spectrum in spectra_files list to use
choose_spectrum_id = 1 # @param {type:"slider", min:0, max:31, step:1}


''' wrap spectrum plotting as a function; allows single or looping plots
  keep in mind that none of these variables are saved
  function's plot isn't closed (doesn't include plt.show()) '''
def PLOT_SPECTRUM(spectrum_id):

  ## load data for chosen spectrum
  # get specified spectrum data
  target_label = spectra_files[spectrum_id][5:-5]  # plate-mjd-fiberid of SDSS target
  spectrum = spectra_fits[spectrum_id]  # HDUL
  COADD = spectrum[1]  # HDU 1 (extname COADD), "Coadded Spectrum from spPlate"

  # put relevant values into 1d arrays
  headers = spectrum[0].header
  wavelengths_obs = 10**COADD.data['loglam']
  targ_flux = COADD.data['flux']
  model_flux = COADD.data['model']


  ## comprehend data
  # filter out negative raw fluxes, as they drag galaxy plots down
  cut_off = -3
  rm_idx = [i for i in range(len(targ_flux)) if targ_flux[i] < cut_off]
  wavelengths_obs = np.delete(wavelengths_obs, rm_idx)
  targ_flux = np.delete(targ_flux, rm_idx)
  model_flux = np.delete(model_flux, rm_idx)

  # redshift correction
  redshift = headers['Z']  # from SDSS database, not from earlier calculations
  wavelengths_src = wavelengths_obs / (1+redshift)

  # choose line set by the class of the target
  if headers['CLASS'] == "GALAXY":
    weight = 'galaxy_weight'
    emissions_targ = emissions[gw_em_idx]
    absorptions_targ = absorptions[gw_ab_idx]
  elif headers['CLASS'] == "QSO":
    weight = 'quasar_weight'
    emissions_targ = emissions[qw_em_idx]
    absorptions_targ = None
  else: # probably a star
    emissions_targ = None
    absorptions_targ = None



  ## plot spectrum of selected object
  plt.subplots(1, 1, figsize=(12,5))
  plt.title(f"{headers['SUBCLASS']} {headers['CLASS']} (SDSS ID {target_label})\nRA = {headers['RA']}$\degree$     DEC = {headers['DEC']}$\degree$     z = {redshift}")

  # draw spectrum's flux and best fit
  plt.plot(wavelengths_src, targ_flux, color='grey', linewidth=0.5, label=f'Flux')
  plt.plot(wavelengths_src, model_flux, color='k', linewidth=1, label='Best Fit')


  ## plot spectral line features
  # condition to filter for lines within plotted wavelength range
  def in_lam_range(line):
    val = line['wavelength']
    run = wavelengths_src
    if val > min(run) and val < max(run):
      return True
    else: return False

  # for offsetting the line labels
  top_plot = max(targ_flux)
  bot_plot = min(targ_flux)
  vscale_plot = abs(top_plot-bot_plot)
  xscale_plot = abs(max(wavelengths_src) - min(wavelengths_src))
  x_offset = xscale_plot * 0.003  # move line text markers over slightly

  # mark emission lines
  if type(emissions_targ) != type(None):
    emissions_filt = filter(in_lam_range, emissions_targ)  # choose lines that fit on plot
    plt.plot([], color='blue', linestyle='--', linewidth=0.5, label='Emission Lines')  # add entry to legend
    for i, line in enumerate(emissions_filt):
      strength = 0.1*line[weight]  # how strongly the line should be annotated
      y_offset = bot_plot + vscale_plot * (0.9 + 0.04*(i%3))  # cascade line markers vertically
      plt.axvline(x=line['wavelength'], color='blue', linestyle='--', linewidth=strength, label='_nolegend_')  # plot dashed line
      txt = plt.text(line['wavelength']+x_offset, y_offset, line['species'], color='blue', alpha=strength, verticalalignment='bottom')  # annotate line
      txt.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='w')])  # add white border to annotation for visibility

  # mark absorption lines
  if type(absorptions_targ) != type(None):
    absorptions_filt = filter(in_lam_range, absorptions_targ)  # choose lines that fit on plot
    plt.plot([], color='red', linestyle='--', linewidth=0.5, label='Absorption Lines')  # add entry to legend
    for i, line in enumerate(absorptions_filt):
      strength = 0.3  # how strongly the line should be annotated (each absorption line has weight -1 for galaxies)
      y_offset = bot_plot + vscale_plot * (0.0 + 0.04*(i%3))  # cascade line markers vertically
      plt.axvline(x=line['wavelength'], color='red', linestyle='--', linewidth=strength, label='_nolegend_')  # plot dashed line
      txt = plt.text(line['wavelength']+x_offset, y_offset, line['species'], color='red', alpha=strength, verticalalignment='bottom')  # annotate line
      txt.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='w')])  # add white border to annotation for visibility


  ## plot configurations
  plt.xlabel("Redshift-Corrected Wavelength [Å]")
  plt.ylabel("Flux [$10^{-17} ergs/s/cm^2/\AA$]")
  plt.grid(axis='y', linewidth=0.1)
  #plt.legend(fontsize=8)  # legend often covers something, may not want it


# perform for single plot
PLOT_SPECTRUM(choose_spectrum_id)
plt.show()



### also plot original spectrum, useful for picking values in guessing line features
## load data for chosen spectrum
# get specified spectrum data
target_label = spectra_files[choose_spectrum_id][5:-5]  # plate-mjd-fiberid of SDSS target
spectrum = spectra_fits[choose_spectrum_id]  # HDUL
COADD = spectrum[1]  # HDU 1 (extname COADD), "Coadded Spectrum from spPlate"

# put relevant values into 1d arrays
headers = spectrum[0].header
wavelengths_obs = 10**COADD.data['loglam']
targ_flux = COADD.data['flux']
model_flux = COADD.data['model']

## plot spectrum of selected object
plt.subplots(1, 1, figsize=(12,5))
plt.title(f"Observed {headers['SUBCLASS']} {headers['CLASS']} (SDSS ID {target_label})\nRA = {headers['RA']}$\degree$     DEC = {headers['DEC']}$\degree$")

# draw spectrum's flux and best fit
plt.plot(wavelengths_obs, targ_flux, color='grey', linewidth=0.5, label=f'Flux')
plt.plot(wavelengths_obs, model_flux, color='k', linewidth=1, label='Best Fit')

# plot configurations
plt.xlabel("Redshift-Corrected Wavelength [Å]")
plt.ylabel("Flux [$10^{-17} ergs/s/cm^2/\AA$]")
plt.locator_params(axis='x', nbins=30)  # increase ticks to ease visually choosing line widths
plt.xticks(rotation = -45)
plt.grid(True, 'both')
plt.legend()
plt.show()

In [ ]:
# @title Analyze expected line features  { form-width: "1px" }


# choose spectral feature
# ids 0-35 are emission lines, 36-45 are the absorption lines
line_id = 7  # @param {type:'integer'}
line = lines[line_id]
if line_id < 36: line_type = "emission"
else: line_type = "absorption"
print(line['species'], "line:", line['wavelength'], "Å")

# choose target
spectrum_id = 0  # @param {type:'integer'}


## load data for chosen spectrum
# get specified spectrum data
target_label = spectra_files[spectrum_id][5:-5]  # plate-mjd-fiberid of SDSS target
spectrum = spectra_fits[spectrum_id]  # HDUL
COADD = spectrum[1]  # HDU 1 (extname COADD), "Coadded Spectrum from spPlate"

# put relevant values into 1d arrays
headers = spectrum[0].header
wavelengths_obs = 10**COADD.data['loglam']
targ_flux = COADD.data['flux']
model_flux = COADD.data['model']


## pick out observed feature
# function to find nearest index of an array given an estimated element
def nearest_i(array, target):
  index = np.abs(array - target).argmin()
  return index

# function to limit arrays around the feature
def pick_feature(center, width):
  # convert element into indices of wavelengths array
  low_i = nearest_i(wavelengths_obs, center-width/2)
  high_i = nearest_i(wavelengths_obs, center+width/2)

  # array of wavelengths of feature
  line_lams = wavelengths_obs[low_i:high_i]
  line_tflux = targ_flux[low_i:high_i]
  line_mflux = model_flux[low_i:high_i]

  return line_lams, line_tflux, line_mflux

# guess where the feature is, same units as wavelengths
guess_lam = 4515  # @param {type:'integer'}
line_width = 50  # @param {type:'integer'}
line_lams, line_tflux, line_mflux = pick_feature(guess_lam, line_width)


## fit line curve to a Gaussian
# the Gaussian function
def Gauss(x, estimate, uncertainty, amplitude):
   return amplitude * np.exp(-(x - estimate)**2 / (2 * uncertainty**2))

# initial guess for parameters
init = [guess_lam, np.std(line_lams), np.max(line_mflux)]

# fit data to Gauss
params, covar = curve_fit(Gauss, line_lams, line_mflux, p0=init)

# get optimized parameters
line_center, line_spread, amplitude = params
line_FWHM = 2 * np.sqrt(2*np.log(2)) * line_spread


## calculate distance information
# redshift from observed line's difference from expected
redshift_obs = line_center / line['wavelength'] - 1
redshift_err = line_spread / line['wavelength']


## plot of selected feature
plt.subplots(1, 1, figsize=(9,6))
plt.title(f"SDSS ID {target_label}\nObserved {line_type} feature '{line['species']}'")

# re-center plotted wavelengths using fitted data
line_lams, line_tflux, line_mflux = pick_feature(line_center, line_width)

# draw spectrum's flux and best fit
plt.plot(line_lams, line_tflux, color='grey', linewidth=0.5, label='Flux')
plt.plot(line_lams, line_mflux, color='k', linewidth=1, label='Model Fit')

# plot fitted curve
plt.plot(line_lams, Gauss(line_lams, line_center, line_spread, amplitude), 'g-', label="Gaussian Fit")

# annotate plot with optimization parameters
annotation = f"Mean Wavelength = {line_center:.6} $\pm$ {line_spread:.4} Å\nFull-Width Half Max = {line_FWHM:.4} Å"
offset = min(line_mflux) + 0.2 * (max(line_mflux) - min(line_mflux))
txt = plt.text(line_center, offset, annotation, horizontalalignment='center')
txt.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])  # clear lines incase of overlap

# plot configurations
plt.xlabel("Wavelength [Å]")
plt.ylabel("Flux [$10^{-17} ergs/s/cm^2/\AA$]")
plt.legend()
plt.show()



---



In [ ]:
# @title Plot every target spectrum

for spectrum_id in range(len(spectra_fits)):
  PLOT_SPECTRUM(spectrum_id)
  plt.show()

In [ ]:
# @title Plot every sky spectra
# for seeing how they differ between measurements, I guess

sky_spectra = [spectrum[1].data['sky'] for spectrum in spectra_fits]
sky_wavelengths = [10**spectrum[1].data['loglam'] for spectrum in spectra_fits]

ss = len(sky_spectra)  # count of spectra


low_flux = min([min(flux) for flux in sky_spectra])
high_flux = max([max(flux) for flux in sky_spectra])

low_lam = min([min(lam) for lam in sky_wavelengths])
high_lam = max([max(lam) for lam in sky_wavelengths])


# make combined plot
plt.subplots(1, 1, figsize=(15, 7.5))

plt.plot([], color='r', linewidth=1, label="MJD = 53433")
for i in range(0, 9):
  plt.plot(sky_wavelengths[i], sky_spectra[i], color='r', linewidth=0.1, zorder=3)

plt.plot([], color='g', linewidth=1, label="MJD = 56067, 56072")
for i in range(9, 25):
  plt.plot(sky_wavelengths[i], sky_spectra[i], color='g', linewidth=0.1, zorder=2)

plt.plot([], color='b', linewidth=1, label="MJD = 58191")
for i in range(25, ss):
  plt.plot(sky_wavelengths[i], sky_spectra[i], color='b', linewidth=0.1, zorder=1)

plt.title(f"Overlapped SDSS sky spectra for selection")
plt.xlabel("Wavelength, [Å]")
plt.xlim(low_lam, high_lam)
plt.ylabel("Flux, [$10^{-17} ergs/s/cm^2/\AA$]")
plt.ylim(low_flux, high_flux)
plt.legend()
plt.show()


# plot each spectrum individually
for i in range(ss):
  plt.subplots(1, 1, figsize=(10, 5))
  plt.plot(sky_wavelengths[i], sky_spectra[i], color='k')
  plt.title(f"SDSS ID {spectra_files[i][5:-5]}")
  plt.xlabel("Wavelength, [Å]")
  plt.xlim(low_lam, high_lam)
  plt.ylabel("Flux, [$10^{-17} ergs/s/cm^2/\AA$]")
  plt.ylim(low_flux, high_flux)
  plt.show()



---



In [ ]:
# @title Sample FITs file header information
# for me to get a better idea of how these files are structured

#spectrum = spectra_fits[random.randint(0, 31)]  # HDUL
spectrum = spectra_fits[0]  # HDUL

spectrum.info()

print()

HDU0 = spectrum[0]
HDU1 = spectrum[1]
HDU2 = spectrum[2]
HDU3 = spectrum[3]

print()

print("PRIMARY\n   ", list(HDU0.header), "\n")
print("COADD\n   ", list(HDU1.header), "\n")
print("SPALL\n   ", list(HDU2.header), "\n")
print("SPZLINE\n   ", list(HDU3.header), "\n")

print()

headers = HDU0.header

for header in list(headers):
  print(header, " = ", headers[header])